### 설치

In [10]:
# konlpy 설치
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 4.1 MB/s 
     |████████████████████████████████| 453 kB 38.7 MB/s 


In [11]:
# 라이브러리

from konlpy.tag import Okt # komoran, hannanum, kkma, mecab
okt = Okt()
import os

import numpy as np
import pandas as pd

from datetime import datetime
import json
import re


from tqdm.notebook import tqdm

import pandas as pd
import requests
from bs4 import BeautifulSoup

### 토큰화+벡터화

In [12]:
data = pd.read_excel('/content/불용어 제거 및 라벨링한 데이터.xlsx')
data

,내용,라벨
0,가 사용자 적격 인정 여부 연맹과 노동조합의 대표가 동일하고 주소지가 같으며 통합...,0
1,가 징계해고 사유의 존재 여부 근로자는 공무직으로 서울시 공무직 채용 및 복무 등...,0
2,근로자는 구인공고를 보고 입사지원을 하였고 구인공고에는 1개월 단기 당직의 ...,0
3,근로자는 관리소장이 지시를 거부하는 근로자는 필요 없다 다른 사람을 구할 테...,0
4,가 근로계약의 법적 성격 기간의 정함이 없는 최초 근로계약서 체결 직후 사용자...,0
...,...,...
12631,근로자를 채용하면서 학력 또는 경력을 기재한 이력서의 제출을 요구하는 것은 ...,0
12632,신청인이 냉장사업부장으로 근무하던 기간 중 부하직원들의 횡령사고에 대한 관리감독...,1
12633,되는 경우로 홍OO 및 류OO은 비록 4대 보험에 가입되어 있었으나 근...,2
12634,신청인의 행위는 징계대상이 된다 할 것이나 폭행당사자가 징계위원으로 참석한것은 공정...,1


In [13]:
def tokenizer(raw, pos=["Noun", "Verb","Adjective" ] , stopword = []):
    return [
        word for word, tag in okt.pos(
            raw,
            norm=True,
            stem=True
        )
        if len(word) > 1 and tag in pos and word not in stopword
        ]

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer ,CountVectorizer

vectorize = TfidfVectorizer(
    tokenizer=tokenizer,
    min_df=2,
    sublinear_tf = True
)

In [17]:
x = vectorize.fit_transform(data['내용'])

In [19]:
y = data['라벨']

#교차검증 (case 10)

In [1]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_validate

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_predict
from sklearn import model_selection
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

In [2]:
def make_df(x):
  result = pd.DataFrame(x,index = ['case1', 'case2', 'case3', 'case4', 'case5', 'case6', 'case7', 'case8', 'case9', 'case10'])
  mean = pd.DataFrame(result.mean()).T
  return result.append(mean)

####간단한 요약

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,

  

=> 사이킷런이 upgrade되면서 LogisticRegression에서 solver에 따른 warning 메시지가 나오곤 합니다.

이 solver는  Gradient Descent와 같이 weight 값을 최적화 하는 유형들을 구분한 것입니다. max_iter는 Gradient Descent 방식을 반복해서 몇번 수행할 것인가 인데, 이게 일단 수렴(Convergence)하게 되면 횟수를 늘려도 성능이 거의 달라지지 않습니다.  따라서 500, 1000, 2000을 넣어도 일단 수렴이 된 후에 반복은 성능이 달라지지 않았기 때문에 같은 결과를 나타낸걸로 보시면 될 것 같습니다.

In [21]:
clf = LogisticRegression(max_iter=500)
model_rf = RandomForestClassifier()
svc = LinearSVC()
NB_clf = MultinomialNB()

score1 = cross_val_score(clf, x, y, cv=10 )
score2 = cross_val_score(model_rf, x, y, cv=10 )
score3 = cross_val_score(svc, x, y, cv=10 )
score4 = cross_val_score(NB_clf, x, y, cv=10 )

print('로지스틱 회귀 교차검증 점수 : {:.2f}'.format(score1.mean()))
print('랜덤포레스트 교차검증 점수 : {:.2f}'.format(score2.mean()))
print('SVC 교차검증 점수 : {:.2f}'.format(score3.mean()))
print('나이브 베이즈 교차검증 점수 : {:.2f}'.format(score4.mean()))

로지스틱 회귀 교차검증 점수 : 0.81
랜덤포레스트 교차검증 점수 : 0.79
SVC 교차검증 점수 : 0.80
나이브 베이즈 교차검증 점수 : 0.72


####로지스틱 회귀 모델

In [22]:
#로지스틱 회귀 모델 교차검증
from sklearn.model_selection import cross_validate
clf = LogisticRegression(max_iter=500)
res = cross_validate(clf, x, y, return_train_score=True, cv=10)

make_df(res)

,fit_time,score_time,test_score,train_score
case1,2.905793,0.001808,0.798259,0.899842
case2,3.066602,0.001836,0.858386,0.893422
case3,2.872540,0.001892,0.856013,0.891664
case4,2.774629,0.001745,0.852057,0.895093
case5,4.115154,0.001763,0.840190,0.893510
case6,3.388728,0.001733,0.826741,0.894741
case7,2.950988,0.001676,0.811560,0.898971
case8,2.836214,0.001760,0.787015,0.899587
case9,2.774304,0.001730,0.771971,0.899411
case10,2.624136,0.001582,0.743468,0.902752


#### 랜덤 포레스트 모델

In [23]:
# 랜덤 포레스트 교차검증 
model_rf = RandomForestClassifier()
re = cross_validate(model_rf, x, y, return_train_score=True, cv=10)
make_df(re)

,fit_time,score_time,test_score,train_score
case1,13.875169,0.088527,0.784019,0.998065
case2,13.852121,0.086839,0.844146,0.998065
case3,13.676443,0.091010,0.820411,0.998065
case4,13.514927,0.087668,0.812500,0.998153
case5,13.673219,0.085284,0.799051,0.998065
case6,13.688941,0.088181,0.799051,0.998153
case7,13.602931,0.082651,0.812352,0.998066
case8,13.786988,0.086776,0.764054,0.998154
case9,13.436495,0.083316,0.723674,0.999121
case10,12.892179,0.080111,0.728424,0.999121


####나이브 베이즈 모델

In [24]:
from sklearn.naive_bayes import MultinomialNB
NB_clf = MultinomialNB()
r = cross_validate(NB_clf, x, y, return_train_score=True, cv=10)
make_df(r)


,fit_time,score_time,test_score,train_score
case1,0.011209,0.001147,0.700949,0.791945
case2,0.010670,0.001224,0.731013,0.786933
case3,0.010634,0.001144,0.715190,0.788164
case4,0.010742,0.001165,0.705696,0.786669
case5,0.010254,0.001106,0.719937,0.782360
case6,0.010501,0.001093,0.754747,0.785526
case7,0.010189,0.001026,0.737134,0.784841
case8,0.009783,0.000997,0.696754,0.789326
case9,0.010292,0.001038,0.688044,0.788095
case10,0.009984,0.000988,0.708630,0.790029


####선형 SVC 모델

In [25]:
# LinearSVC모델 

from sklearn import svm
from sklearn.svm import SVC
from sklearn.svm import LinearSVC

l_svc = LinearSVC()
r = cross_validate(l_svc, x, y, return_train_score=True, cv=10)
make_df(r)


# model = LinearSVC(C=1000)
# model.fit(X_train_tfidf, y_train)
# print(model.score(X_train_tfidf, y_train))
# print(model.score(X_test_tfidf, y_test))
    

,fit_time,score_time,test_score,train_score
case1,0.288483,0.001248,0.784019,0.954362
case2,0.301847,0.001303,0.848101,0.949349
case3,0.293550,0.001228,0.850475,0.949613
case4,0.298500,0.001255,0.849684,0.949437
case5,0.299550,0.001212,0.829905,0.947327
case6,0.283572,0.001168,0.812500,0.951636
case7,0.298758,0.001189,0.808393,0.951200
case8,0.286256,0.001208,0.772763,0.952959
case9,0.290564,0.001267,0.758511,0.955421
case10,0.286595,0.001114,0.728424,0.958059


#### SVC 모델

In [26]:
#SVC모델 
svc = SVC()
rr = cross_validate(svc, x, y, return_train_score=True, cv=10)
make_df(rr)


,fit_time,score_time,test_score,train_score
case1,65.735963,6.723481,0.804589,0.978632
case2,68.243932,6.500926,0.863133,0.975818
case3,66.895622,6.913397,0.864715,0.976433
case4,67.727198,6.468618,0.855222,0.975818
case5,67.789250,6.220731,0.849684,0.977313
case6,67.660158,6.348387,0.832278,0.977577
case7,67.628227,6.206877,0.830562,0.977578
case8,67.184154,6.222018,0.791766,0.977666
case9,66.727684,6.141267,0.782264,0.979249
case10,67.915151,5.574940,0.743468,0.980040
